## This Notebook is about Applied Data Science Capstone Project 

In [107]:
import pandas as pd
import numpy as np

In [108]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Week 3 from here

# Importing Libraries

In [109]:
import lxml.html as lh
import requests

In [110]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

# Getting columns names

In [111]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



# Getting data

In [112]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

# Making DataFrame

In [113]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# Removing records with Borough not assigned

In [114]:
df=df[df.Borough!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


# Replacing Not assigned Neighbourhoods with Borough

In [115]:
indexes=df['Neighbourhood\n']=='Not assigned\n'
df.loc[indexes,'Neighbourhood\n']=df.loc[indexes,'Borough']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


# Combining Neighbourhoods 

In [116]:
df = df.groupby(df['Postcode']).aggregate({'Borough': 'first', \
                                               'Neighbourhood\n': 'sum'})
df = df.reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge\nMalvern\n
1,M1C,Scarborough,Highland Creek\nRouge Hill\nPort Union\n
2,M1E,Scarborough,Guildwood\nMorningside\nWest Hill\n
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


# Shape of DataFrame

In [117]:
df.shape

(103, 3)

# Reading Lattitude and Longitude

In [118]:
crdnts=pd.read_csv('Geospatial_Coordinates.csv')
crdnts.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Joining both DataFrames

In [119]:
df=df.set_index('Postcode').join(crdnts.set_index('Postal Code'))
df = df.reset_index()
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge\nMalvern\n,43.806686,-79.194353
1,M1C,Scarborough,Highland Creek\nRouge Hill\nPort Union\n,43.784535,-79.160497
2,M1E,Scarborough,Guildwood\nMorningside\nWest Hill\n,43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge\nMalvern\n,43.806686,-79.194353
1,M1C,Scarborough,Highland Creek\nRouge Hill\nPort Union\n,43.784535,-79.160497
2,M1E,Scarborough,Guildwood\nMorningside\nWest Hill\n,43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476
